Setup rendering dependencies for Google Colaboratory.

In [7]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install 'gym[classic_control]' > /dev/null 2>&1
!brew install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Install d3rlpy!

In [8]:
!pip install d3rlpy

Setup cartpole dataset.

In [9]:
from d3rlpy.datasets import get_cartpole

# get CartPole dataset
dataset, env = get_cartpole()

/Users/jackmckew/miniconda3/envs/d3/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


Setup data-driven deep reinforcement learning algorithm.

In [10]:
from d3rlpy.algos import DiscreteCQL
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split

# setup CQL algorithm
cql = DiscreteCQL(use_gpu=False)

# split train and test episodes
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

# start training
cql.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=1,
        scorers={
            'environment': evaluate_on_environment(env), # evaluate with CartPol-v0 environment
            'advantage': discounted_sum_of_advantage_scorer, # smaller is better
            'td_error': td_error_scorer, # smaller is better
            'value_scale': average_value_estimation_scorer # smaller is better
        })

2023-04-14 23:55:34 [debug    ] RoundIterator is selected.
2023-04-14 23:55:34 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20230414235534
2023-04-14 23:55:34 [debug    ] Building models...
2023-04-14 23:55:34 [debug    ] Models have been built.
2023-04-14 23:55:34 [info     ] Parameters are saved to d3rlpy_logs/DiscreteCQL_20230414235534/params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_update_interval': 8000, 'use_gpu': None, 'algorithm': 'DiscreteCQL', 'observation

Epoch 1/1:   0%|          | 0/2473 [00:00<?, ?it/s]

AttributeError: 'list' object has no attribute 'to'

Setup rendering utilities for Google Colaboratory.

In [11]:
import glob
import io
import base64

from gym.wrappers import RecordVideo
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display

# start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

# play recorded video
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''
            <video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

Record video!

In [12]:
# wrap RecordVideo wrapper
env = RecordVideo(env, './video')

# evaluate
evaluate_on_environment(env)(cql)

DependencyNotInstalled: MoviePy is not installed, run `pip install moviepy`

Let's see how it works!

In [ ]:
show_video()